In [2]:
!pip install finance-datareader

In [18]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [40]:
# 삼성전자 주가 데이터 가져오기 (최근 32일)
stock_data = fdr.DataReader('005930', start=pd.Timestamp.now() - pd.Timedelta('60D'))
stock_data = stock_data.iloc[-32:]

In [41]:
# 이미지 크기 설정
width, height = 96, 96

# High, Low, Close, Volume 데이터 정규화
high_prices = stock_data['High'].values
low_prices = stock_data['Low'].values
close_prices = stock_data['Close'].values
volume_data = stock_data['Volume'].values


high_prices_norm = (high_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
low_prices_norm = (low_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
close_prices_norm = (close_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
volume_data_norm = height * (volume_data) / (2 * (np.max(volume_data)))


In [49]:
def alpha_blend(color1, color2, alpha):
    return [
        int((color1[0] * alpha + color2[0] * (1 - alpha))),
        int((color1[1] * alpha + color2[1] * (1 - alpha))),
        int((color1[2] * alpha + color2[2] * (1 - alpha))),
        255,
    ]

# 96x96 이미지 생성
img = np.zeros((height, width, 4), dtype=np.uint8) # 4 channels for RGBA

for day in range(32):
    high_price = int(high_prices_norm[day] * (height - 1))
    low_price = int(low_prices_norm[day] * (height - 1))
    close_price = int(close_prices_norm[day] * (height - 1))
    open_price = int(
        (
            (stock_data['Open'].values[day] - np.min(low_prices))
            / (np.max(high_prices) - np.min(low_prices))
        )
        * (height - 1)
    )
    volume = int(volume_data_norm[day])

    x_start = day * 3

    # 막대 왼쪽, 오른쪽 그리기 (시가, 종가 범위)
    for x in range(x_start, x_start + 3):
        if close_prices[day] > stock_data['Open'].values[day]:
            color = [0, 0, 255, 255]  # 상승: 파랑
        else:
            color = [255, 0, 0, 255]  # 하락: 빨강
        for y in range(min(open_price, close_price), max(open_price, close_price) + 1):
            img[y, x] = color

    # 꼬리 그리기 (High, Low 범위)
    for x in range(x_start + 1, x_start + 2):
        for y in range(low_price, high_price + 1):
            if img[y, x][0] != 255 and img[y, x][2] != 255:
                img[y, x] = [200, 200, 200, 255]

    # 거래량 그리기
    for x in range(x_start, x_start + 3):
        overlapped = height - volume
        if overlapped >= 0 and overlapped < height:
            img[overlapped:, x] = [
                alpha_blend(img[y, x], [128, 128, 128, 255], 0.45)
                for y in range(overlapped, height)
            ]

# 이미지 저장
img = Image.fromarray(img, 'RGBA')
img.save('samsung_stock_data_image.png')
img.show()

IndexError: ignored